## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users\kkubi/Class Repo/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.90,76,1,11.36,clear sky
1,1,Imeni Babushkina,RU,59.7573,43.1285,16.86,88,100,11.14,overcast clouds
2,2,Norman Wells,CA,65.2820,-126.8329,-18.36,69,75,3.44,light snow
3,3,Ancud,CL,-41.8697,-73.8203,49.53,92,34,4.72,scattered clouds
4,4,Bredasdorp,ZA,-34.5322,20.0403,51.28,77,22,3.24,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("On your trip, what is the coldest temperature you want it to be? "))
max_temp = float(input("On your trip, what is the hotest temperature you want it to be?" ))

On your trip, what is the coldest temperature you want it to be? 70
On your trip, what is the hotest temperature you want it to be?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.90,76,1,11.36,clear sky
7,7,Busselton,AU,-33.6500,115.3333,71.15,42,0,10.04,clear sky
12,12,Mabay,PH,6.0577,124.4666,85.86,69,100,4.38,overcast clouds
22,22,Vaini,TO,-21.2000,-175.2000,84.36,79,40,5.75,scattered clouds
28,28,Road Town,VG,18.4167,-64.6167,78.01,73,32,13.78,scattered clouds
29,29,Meulaboh,ID,4.1363,96.1285,80.53,78,100,2.48,overcast clouds
34,34,Hithadhoo,MV,-0.6000,73.0833,81.12,79,100,17.96,overcast clouds
36,36,Sambava,MG,-14.2667,50.1667,76.42,78,64,4.76,light rain
39,39,Kapaa,US,22.0752,-159.3190,80.58,78,90,1.01,overcast clouds
45,45,Tefe,BR,-3.3542,-64.7114,72.79,99,88,2.62,very heavy rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                229
City                   229
Country                229
Lat                    229
Lng                    229
Max Temp               229
Humidity               229
Cloudiness             229
Wind Speed             229
Current Description    229
dtype: int64

In [6]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
preferred_cities_df.notnull().sum()

City_ID                229
City                   229
Country                229
Lat                    229
Lng                    229
Max Temp               229
Humidity               229
Cloudiness             229
Wind Speed             229
Current Description    229
dtype: int64

### 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
###       I did this step in STEP #3 by placing dropna() outside the loc method's square brackets.

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.90,clear sky,-23.1203,-134.9692,
7,Busselton,AU,71.15,clear sky,-33.6500,115.3333,
12,Mabay,PH,85.86,overcast clouds,6.0577,124.4666,
22,Vaini,TO,84.36,scattered clouds,-21.2000,-175.2000,
28,Road Town,VG,78.01,scattered clouds,18.4167,-64.6167,
29,Meulaboh,ID,80.53,overcast clouds,4.1363,96.1285,
34,Hithadhoo,MV,81.12,overcast clouds,-0.6000,73.0833,
36,Sambava,MG,76.42,light rain,-14.2667,50.1667,
39,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,
45,Tefe,BR,72.79,very heavy rain,-3.3542,-64.7114,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
         
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.90,clear sky,-23.1203,-134.9692,People ThankYou
7,Busselton,AU,71.15,clear sky,-33.6500,115.3333,Observatory Guest House
12,Mabay,PH,85.86,overcast clouds,6.0577,124.4666,Bautista Lodging House
22,Vaini,TO,84.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
28,Road Town,VG,78.01,scattered clouds,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
...,...,...,...,...,...,...,...
661,San Luis,AR,70.34,clear sky,-33.2950,-66.3356,Huarpes Hotel
662,Chokwe,MZ,71.44,overcast clouds,-24.5333,32.9833,Complexo Sonho Real
665,Douentza,ML,73.44,clear sky,14.9951,-2.9517,Ogomono Doumbo
669,Gawler,AU,78.10,light rain,-34.6000,138.7333,Eagle Foundry Bed & Breakfast


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.90,clear sky,-23.1203,-134.9692,People ThankYou
7,Busselton,AU,71.15,clear sky,-33.6500,115.3333,Observatory Guest House
12,Mabay,PH,85.86,overcast clouds,6.0577,124.4666,Bautista Lodging House
22,Vaini,TO,84.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
28,Road Town,VG,78.01,scattered clouds,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
...,...,...,...,...,...,...,...
661,San Luis,AR,70.34,clear sky,-33.2950,-66.3356,Huarpes Hotel
662,Chokwe,MZ,71.44,overcast clouds,-24.5333,32.9833,Complexo Sonho Real
665,Douentza,ML,73.44,clear sky,14.9951,-2.9517,Ogomono Doumbo
669,Gawler,AU,78.10,light rain,-34.6000,138.7333,Eagle Foundry Bed & Breakfast


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Weather description</dt><dd>{Current Description}</dd>
    <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))